# 解凍

In [1]:
import shutil

shutil.unpack_archive('/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/DrugAtFDA-20241206T090805Z-001.zip', '/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace')

In [13]:
import os

dir_path = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_raw"
files = os.listdir(dir_path)

for file in files:
    if file.endswith(".zip"):
        file_path = os.path.join(dir_path, file)
        shutil.unpack_archive(file_path, '/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_defreeze')

# DataFrame 化
https://qiita.com/spssfun2017/items/937b6bd5ea81ea7974ad が神

## JSONファイルのpandasDF化関数（完成版）
リスト構造が分解されて横に長くなるので不使用

In [5]:
import pandas as pd
import json

# フラット化

def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        # 列名の生成
        new_key = parent_key + sep + k if parent_key else k
        # 辞書型項目のフラット化
        if isinstance(v, dict):
            items.extend(flatten(v, new_key, sep=sep).items())
        # リスト項目のフラット化
        elif isinstance(v, list):
            new_key_tmp = new_key
            for i, elm in enumerate(v):
                new_key = new_key_tmp + sep + str(i)
                # リストの中の辞書
                if isinstance(elm, dict):
                    items.extend(flatten(elm, new_key, sep=sep).items())
                # 単なるリスト
                else:
                    items.append((new_key, elm))
        # 値追加
        else:
            items.append((new_key, v))
    return dict(items)


def flattenJsonFile(jsonfile, rowsroot, sep='.'):
    """
    JSONファイルを読み込み2次元のpandas DataFrameに変換する

    Parameters
    ----------
    jsonfile : string
        JSONファイルパス
    rowsroot : string
        フラット化するルートエレメント名。トップからでいい場合は空文字を入力する
    sep : string
        ノーマライズされていないエレメントを区切る文字

    Returns
    -------
    df : pandas.DataFrame
        フラット化されたpandas DataFrame
    """
   # JSONファイルを読込
    with open(jsonfile, encoding='utf-8') as f:
        d = json.load(f)

    # df化したい辞書リストのルート項目を指定
    if rowsroot != '':
        d = d[rowsroot]

    # フラット化
    dlist = []
    for di in d:
        dlist.append(flatten(di, sep=sep))

    # フラット化された辞書をpandasデータフレームに変換
    return pd.DataFrame.from_dict(dlist)

In [16]:
import os

input_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_defreeze"
output_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_raw_df"
files = os.listdir(input_dir)

for file in files:
    input_file = os.path.join(input_dir, file)
    df = flattenJsonFile(input_file, "results")
    file_tsv = os.path.splitext(file)[0] + ".tsv"
    output_file = os.path.join(output_dir, file_tsv)
    df.to_csv(output_file, sep="\t", index=False)

In [ ]:
df = pd.read_csv("/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_list/drug-label-0012-of-0012.tsv", sep="\t")
df.head()

,spl_product_data_elements.0,description.0,clinical_pharmacology.0,indications_and_usage.0,contraindications.0,precautions.0,adverse_reactions.0,overdosage.0,dosage_and_administration.0,spl_unclassified_section.0,...,openfda.product_ndc.29,pharmacodynamics.1,openfda.route.2,boxed_warning_table.0,information_for_patients_table.1,openfda.upc.11,pregnancy_or_breast_feeding.1,pregnancy_or_breast_feeding.2,pregnancy_or_breast_feeding.3,pregnancy_or_breast_feeding.4
0,NYSTATIN NYSTATIN NYSTATIN NYSTATIN ALCOHOL SU...,DESCRIPTION Nystatin is obtained from Streptom...,CLINICAL PHARMACOLOGY Nystatin acts by binding...,INDICATIONS AND USAGE Nystatin oral suspension...,CONTRAINDICATIONS Nystatin is contraindicated ...,PRECAUTIONS General Discontinue treatment with...,ADVERSE REACTIONS To report SUSPECTED ADVERSE ...,OVERDOSAGE Oral doses of nystatin in excess of...,"DOSAGE AND ADMINISTRATION Infants: 2 mL (200,0...",CAUTION The Packaging of This Product Contains...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Vilazodone Hydrochloride Vilazodone Hydrochlor...,11 DESCRIPTION Vilazodone hydrochloride tablet...,12 CLINICAL PHARMACOLOGY 12.1 Mechanism of act...,1 INDICATIONS AND USAGE Vilazodone hydrochlori...,4 CONTRAINDICATIONS Concomitant use of monoami...,NaN,6 ADVERSE REACTIONS The following adverse reac...,10 OVERDOSAGE There is limited clinical trial ...,2 DOSAGE AND ADMINISTRATION Recommended target...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Azithromycin Dihydrate Azithromycin Dihydrate ...,"11 DESCRIPTION Azithromycin Tablets, USP conta...",12 CLINICAL PHARMACOLOGY 12.1 Mechanism of Act...,1 INDICATIONS AND USAGE Azithromycin is a macr...,4 CONTRAINDICATIONS • Patients with known hype...,NaN,6 ADVERSE REACTIONS Most common adverse reacti...,10 OVERDOSAGE Adverse reactions experienced at...,2 DOSAGE AND ADMINISTRATION • Adult Patients (...,"Manufactured by: CSPC Ouyi Pharmaceutical Co.,...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Benzonatate Benzonatate BENZONATATE BENZONATAT...,"DESCRIPTION Benzonatate, a non-narcotic oral a...",CLINICAL PHARMACOLOGY Benzonatate Capsule acts...,INDICATIONS AND USAGE Benzonatate Capsule is i...,CONTRAINDICATIONS Hypersensitivity to benzonat...,PRECAUTIONS Benzonatate is chemically related ...,ADVERSE REACTIONS Potential Adverse Reactions ...,OVERDOSAGE Intentional and unintentional overd...,DOSAGE AND ADMINISTRATION Adults and Children ...,Rev. 2616 Manufactured by: CSPC-NBP Pharmaceut...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Tiger Balm Camphor (Synthetic), Menthol, and C...",NaN,NaN,Uses For temporary relief of pains.,NaN,NaN,NaN,NaN,Directions Adults and children 12 years and ov...,Drug Facts,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## フラット化とルートの移動のみ

In [25]:
import pandas as pd
import json

def flattenJsonFile(jsonfile, rowsroot, sep='.'):
    """
    JSONファイルを読み込み2次元のpandas DataFrameに変換する

    Parameters
    ----------
    jsonfile : string
        JSONファイルパス
    rowsroot : string
        フラット化するルートエレメント名。トップからでいい場合は空文字を入力する
    sep : string
        ノーマライズされていないエレメントを区切る文字

    Returns
    -------
    df : pandas.DataFrame
        フラット化されたpandas DataFrame
    """
   # JSONファイルを読込
    with open(jsonfile, encoding='utf-8') as f:
        d = json.load(f)

    # df化したい辞書リストのルート項目を指定
    if rowsroot != '':
        d = d[rowsroot]

    df = pd.json_normalize(d)

    # フラット化された辞書をpandasデータフレームに変換
    return df

In [26]:
import os

input_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_defreeze"
output_dir = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_raw_df"
files = os.listdir(input_dir)

for file in files:
    input_file = os.path.join(input_dir, file)
    df = flattenJsonFile(input_file, "results")
    file_tsv = os.path.splitext(file)[0] + ".tsv"
    output_file = os.path.join(output_dir, file_tsv)
    df.to_csv(output_file, sep="\t", index=False)

In [27]:
df = pd.read_csv("/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_raw_df/drug-label-0012-of-0012.tsv", sep="\t")
df.head()

,spl_product_data_elements,description,clinical_pharmacology,indications_and_usage,contraindications,precautions,adverse_reactions,overdosage,dosage_and_administration,spl_unclassified_section,...,health_care_provider_letter_table,dosage_forms_and_strengths_table,storage_and_handling_table,user_safety_warnings,information_for_patients_table,geriatric_use_table,spl_indexing_data_elements,patient_medication_information_table,intended_use_of_the_device,boxed_warning_table
0,['NYSTATIN NYSTATIN NYSTATIN NYSTATIN ALCOHOL ...,['DESCRIPTION Nystatin is obtained from Strept...,['CLINICAL PHARMACOLOGY Nystatin acts by bindi...,['INDICATIONS AND USAGE Nystatin oral suspensi...,['CONTRAINDICATIONS Nystatin is contraindicate...,['PRECAUTIONS General Discontinue treatment wi...,['ADVERSE REACTIONS To report SUSPECTED ADVERS...,['OVERDOSAGE Oral doses of nystatin in excess ...,['DOSAGE AND ADMINISTRATION Infants: 2 mL (200...,['CAUTION The Packaging of This Product Contai...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['Vilazodone Hydrochloride Vilazodone Hydrochl...,['11 DESCRIPTION Vilazodone hydrochloride tabl...,"[""12 CLINICAL PHARMACOLOGY 12.1 Mechanism of a...",['1 INDICATIONS AND USAGE Vilazodone hydrochlo...,['4 CONTRAINDICATIONS Concomitant use of monoa...,NaN,['6 ADVERSE REACTIONS The following adverse re...,['10 OVERDOSAGE There is limited clinical tria...,['2 DOSAGE AND ADMINISTRATION Recommended targ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['Azithromycin Dihydrate Azithromycin Dihydrat...,"['11 DESCRIPTION Azithromycin Tablets, USP con...",['12 CLINICAL PHARMACOLOGY 12.1 Mechanism of A...,['1 INDICATIONS AND USAGE Azithromycin is a ma...,['4 CONTRAINDICATIONS • Patients with known hy...,NaN,['6 ADVERSE REACTIONS Most common adverse reac...,['10 OVERDOSAGE Adverse reactions experienced ...,['2 DOSAGE AND ADMINISTRATION • Adult Patients...,['Manufactured by: CSPC Ouyi Pharmaceutical Co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['Benzonatate Benzonatate BENZONATATE BENZONAT...,"['DESCRIPTION Benzonatate, a non-narcotic oral...",['CLINICAL PHARMACOLOGY Benzonatate Capsule ac...,['INDICATIONS AND USAGE Benzonatate Capsule is...,['CONTRAINDICATIONS Hypersensitivity to benzon...,['PRECAUTIONS Benzonatate is chemically relate...,['ADVERSE REACTIONS Potential Adverse Reaction...,['OVERDOSAGE Intentional and unintentional ove...,['DOSAGE AND ADMINISTRATION Adults and Childre...,['Rev. 2616 Manufactured by: CSPC-NBP Pharmace...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['Tiger Balm Camphor (Synthetic), Menthol, and...",NaN,NaN,['Uses For temporary relief of pains.'],NaN,NaN,NaN,NaN,['Directions Adults and children 12 years and ...,"['Drug Facts', 'Manufactured by Haw Par Health...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# ヘッダーの中身を見ていらない列を削除する

フィールドの説明は [ここ](https://open.fda.gov/apis/drug/label/searchable-fields/) から見られる。  
残す・除外する列についての情報は、fdalabel_columns.csv に。

In [6]:
# 全部の列の確認
import os
import csv

folder_path = "/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/"

headers_list = []

for filename in os.listdir(folder_path):
    if filename.endswith(".tsv"):
        file_path = os.path.join(folder_path, filename)
        with open(file_path, 'r', encoding='utf-8') as file:
            reader = csv.reader(file, delimiter='\t')
            headers = next(reader)  # 最初の行を取得
            headers_list.append((filename, headers))

for file, headers in headers_list:
    print(f"File: {file}")
    print(f"Headers: {headers}")
    print()

File: drug-label-0004-of-0012.tsv
Headers: ['spl_product_data_elements', 'spl_unclassified_section', 'active_ingredient', 'purpose', 'indications_and_usage', 'warnings', 'keep_out_of_reach_of_children', 'dosage_and_administration', 'inactive_ingredient', 'package_label_principal_display_panel', 'set_id', 'id', 'effective_time', 'version', 'do_not_use', 'ask_doctor', 'ask_doctor_or_pharmacist', 'stop_use', 'pregnancy_or_breast_feeding', 'storage_and_handling', 'questions', 'boxed_warning', 'description', 'clinical_pharmacology', 'clinical_pharmacology_table', 'drug_interactions', 'clinical_studies', 'clinical_studies_table', 'contraindications', 'precautions', 'general_precautions', 'information_for_patients', 'laboratory_tests', 'drug_and_or_laboratory_test_interactions', 'carcinogenesis_and_mutagenesis_and_impairment_of_fertility', 'pregnancy', 'nursing_mothers', 'pediatric_use', 'geriatric_use', 'adverse_reactions', 'overdosage', 'how_supplied', 'how_supplied_table', 'spl_patient_pac

In [7]:
# 列名をマージ
result = list(set(item for _, sublist in headers_list for item in sublist))

print(result)

['spl_medguide', 'dosage_forms_and_strengths', 'spl_unclassified_section_table', 'use_in_specific_populations_table', 'openfda.route', 'animal_pharmacology_and_or_toxicology', 'geriatric_use', 'use_in_specific_populations', 'components_table', 'pregnancy_or_breast_feeding', 'information_for_patients_table', 'abuse_table', 'drug_interactions_table', 'drug_interactions', 'geriatric_use_table', 'teratogenic_effects_table', 'other_safety_information', 'precautions_table', 'storage_and_handling', 'clinical_pharmacology_table', 'pharmacodynamics', 'environmental_warning', 'troubleshooting', 'statement_of_identity_table', 'pediatric_use', 'disposal_and_waste_handling', 'pregnancy_or_breast_feeding_table', 'nursing_mothers_table', 'how_supplied', 'package_label_principal_display_panel_table', 'patient_medication_information', 'accessories', 'warnings_and_cautions', 'mechanism_of_action_table', 'dosage_and_administration', 'openfda.application_number', 'health_claim', 'controlled_substance', 'p

In [8]:
# 重複無しの列名がいくつあるか確認
len(result)

179

In [10]:
# 各ファイルの列数が重複無しのすべての列数よりも少ないことを確認
df = pd.read_csv("/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0012-of-0012.tsv", sep="\t")
df.head()

,spl_product_data_elements,description,clinical_pharmacology,indications_and_usage,contraindications,precautions,adverse_reactions,overdosage,dosage_and_administration,spl_unclassified_section,...,health_care_provider_letter_table,dosage_forms_and_strengths_table,storage_and_handling_table,user_safety_warnings,information_for_patients_table,geriatric_use_table,spl_indexing_data_elements,patient_medication_information_table,intended_use_of_the_device,boxed_warning_table
0,['NYSTATIN NYSTATIN NYSTATIN NYSTATIN ALCOHOL ...,['DESCRIPTION Nystatin is obtained from Strept...,['CLINICAL PHARMACOLOGY Nystatin acts by bindi...,['INDICATIONS AND USAGE Nystatin oral suspensi...,['CONTRAINDICATIONS Nystatin is contraindicate...,['PRECAUTIONS General Discontinue treatment wi...,['ADVERSE REACTIONS To report SUSPECTED ADVERS...,['OVERDOSAGE Oral doses of nystatin in excess ...,['DOSAGE AND ADMINISTRATION Infants: 2 mL (200...,['CAUTION The Packaging of This Product Contai...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,['Vilazodone Hydrochloride Vilazodone Hydrochl...,['11 DESCRIPTION Vilazodone hydrochloride tabl...,"[""12 CLINICAL PHARMACOLOGY 12.1 Mechanism of a...",['1 INDICATIONS AND USAGE Vilazodone hydrochlo...,['4 CONTRAINDICATIONS Concomitant use of monoa...,NaN,['6 ADVERSE REACTIONS The following adverse re...,['10 OVERDOSAGE There is limited clinical tria...,['2 DOSAGE AND ADMINISTRATION Recommended targ...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['Azithromycin Dihydrate Azithromycin Dihydrat...,"['11 DESCRIPTION Azithromycin Tablets, USP con...",['12 CLINICAL PHARMACOLOGY 12.1 Mechanism of A...,['1 INDICATIONS AND USAGE Azithromycin is a ma...,['4 CONTRAINDICATIONS • Patients with known hy...,NaN,['6 ADVERSE REACTIONS Most common adverse reac...,['10 OVERDOSAGE Adverse reactions experienced ...,['2 DOSAGE AND ADMINISTRATION • Adult Patients...,['Manufactured by: CSPC Ouyi Pharmaceutical Co...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['Benzonatate Benzonatate BENZONATATE BENZONAT...,"['DESCRIPTION Benzonatate, a non-narcotic oral...",['CLINICAL PHARMACOLOGY Benzonatate Capsule ac...,['INDICATIONS AND USAGE Benzonatate Capsule is...,['CONTRAINDICATIONS Hypersensitivity to benzon...,['PRECAUTIONS Benzonatate is chemically relate...,['ADVERSE REACTIONS Potential Adverse Reaction...,['OVERDOSAGE Intentional and unintentional ove...,['DOSAGE AND ADMINISTRATION Adults and Childre...,['Rev. 2616 Manufactured by: CSPC-NBP Pharmace...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"['Tiger Balm Camphor (Synthetic), Menthol, and...",NaN,NaN,['Uses For temporary relief of pains.'],NaN,NaN,NaN,NaN,['Directions Adults and children 12 years and ...,"['Drug Facts', 'Manufactured by Haw Par Health...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
import pandas as pd

df = pd.read_csv('/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0009-of-0012.tsv', sep="\t")

column_data = df['residue_warning']

column_data_no_missing = column_data.dropna()

print(column_data_no_missing)

7917    ['Other Information Protect the product in thi...
Name: residue_warning, dtype: object


/tmp/ipykernel_2723934/100456593.py:3: DtypeWarning: Columns (68,112,116,117,119,120,121,126,128,129,130,132,135,138,139,140,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0009-of-0012.tsv', sep="\t")


In [12]:
df = pd.read_csv("/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0001-of-0012.tsv", sep="\t")
df.head()

/tmp/ipykernel_2723934/2806970969.py:1: DtypeWarning: Columns (103,104,117,118,119,121,122,123,126,127,129,130,135,139,140,141,142,143,144,146,147,148,149,150,151,152,153,154,155,156,157,158,159) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_flatten_partly/drug-label-0001-of-0012.tsv", sep="\t")


,spl_product_data_elements,boxed_warning,description,description_table,clinical_pharmacology,indications_and_usage,contraindications,warnings,precautions,adverse_reactions,...,ask_doctor_table,pregnancy_or_breast_feeding_table,ask_doctor_or_pharmacist_table,animal_pharmacology_and_or_toxicology_table,information_for_owners_or_caregivers_table,health_care_provider_letter,troubleshooting,health_care_provider_letter_table,risks_table,veterinary_indications
0,['Hydrocodone Bitartrate and Acetaminophen Hyd...,"['WARNING: ADDICTION, ABUSE, AND MISUSE; RISK ...","[""DESCRIPTION Hydrocodone bitartrate and aceta...","['<table width=""100%""><caption/><tbody><tr><td...",['CLINICAL PHARMACOLOGY Mechanism of Action Hy...,['INDICATIONS AND USAGE Hydrocodone bitartrate...,['CONTRAINDICATIONS Hydrocodone bitartrate and...,"['WARNINGS Addiction, Abuse, and Misuse Hydroc...",['PRECAUTIONS Risks of Driving and Operating M...,['ADVERSE REACTIONS The following adverse reac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"['Dextroamphetamine Saccharate, Amphetamine As...","[""WARNING: ABUSE, MISUSE, AND ADDICTION Dextro...",['DESCRIPTION A single-entity amphetamine prod...,"['<table width=""75%""><col width=""24%"" align=""l...","[""CLINICAL PHARMACOLOGY Pharmacodynamics Amphe...",['INDICATIONS AND USAGE Dextroamphetamine Sacc...,['CONTRAINDICATIONS In patients known to be hy...,"[""WARNINGS Abuse, Misuse, and Addiction Dextro...","[""PRECAUTIONS Information for Patients Advise ...",['ADVERSE REACTIONS Cardiovascular Palpitation...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,['Hydrocodone Bitartrate and Acetaminophen Hyd...,"['WARNING: ADDICTION, ABUSE, AND MISUSE;RISK E...","[""DESCRIPTION Hydrocodone bitartrate and aceta...",NaN,['CLINICAL PHARMACOLOGY Mechanism of Action Hy...,['INDICATIONS AND USAGE Hydrocodone bitartrate...,['CONTRAINDICATIONS Hydrocodone Bitartrate and...,"['WARNINGS Addiction, Abuse, and Misuse Hydroc...",['PRECAUTIONS Risks of Driving and Operating M...,['ADVERSE REACTIONS The following adverse reac...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,['Meloxicam MELOXICAM MELOXICAM MELOXICAM SILI...,['WARNING: RISK OF SERIOUS CARDIOVASCULAR AND ...,"['11 DESCRIPTION Meloxicam oral suspension, US...",NaN,"[""12 CLINICAL PHARMACOLOGY 12.1 Mechanism of A...",['1 INDICATIONS AND USAGE Meloxicam oral suspe...,['4 CONTRAINDICATIONS Meloxicam oral suspensio...,NaN,NaN,['6 ADVERSE REACTIONS The following adverse re...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,['Buprenorphine and Naloxone buprenorphine and...,NaN,['11 DESCRIPTION Buprenorphine and naloxone su...,NaN,['12 CLINICAL PHARMACOLOGY 12.1 Mechanism of A...,['1 INDICATIONS AND USAGE Buprenorphine and na...,['4 CONTRAINDICATIONS Buprenorphine and naloxo...,NaN,NaN,['6 ADVERSE REACTIONS The following serious ad...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# 欠損値の多い列を探す

In [100]:
import sqlite3
import os
import pandas as pd

folder_path = '/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/fdalabel_db'
db_files = [f for f in os.listdir(folder_path) if f.endswith('.db')]

all_missing_data = []

for db_file in db_files:
    db_path = os.path.join(folder_path, db_file)
    conn = sqlite3.connect(db_path)
    
    df = pd.read_sql(f"SELECT * FROM raw", conn)
        
    # 各カラムの欠損値数を計算
    missing_values = df.isnull().sum()
        
    # 欠損値が多い順に並べ替え
    missing_values_sorted = missing_values.sort_values(ascending=False)
        
    for column, missing_count in missing_values_sorted.items():
        all_missing_data.append({
            'DB File': db_file,
            'Column': column,
            'Missing Values': missing_count
        })
    
    conn.close()

missing_data_df = pd.DataFrame(all_missing_data)

missing_data_df_sorted = missing_data_df.sort_values(by='Missing Values', ascending=False)

missing_data_df_sorted.to_csv('/workspace/ssd4t/yoshikawa/fdalabel_handler/workspace/missing_values.csv', index=False)